In [ ]:
#------------------------------------------------------------------------------------
# Llama index를 활요한 GPT 인덱싱 구축
# 출처 : https://gpt-index.readthedocs.io/en/latest/getting_started/starter_example.html
#------------------------------------------------------------------------------------

#!pip install openai
#!pip install llama-index

# 환경변수로 OPENAI_API_KEY를 지정해야 함.
import os
os.environ['OPENAI_API_KEY'] = 'sk-xxx'

In [ ]:
# 파일들이 있는 폴더를 지정해서 파일로딩함.
from llama_index import SimpleDirectoryReader

folder_path = 'data'
documents = SimpleDirectoryReader(folder_path).load_data()

In [ ]:
#---------------------------------------------------------------------------
# 시맨틱 검색 예제 
#---------------------------------------------------------------------------
# - 기본 모델은 text-davinci-003 임.
from llama_index import GPTSimpleVectorIndex
index = GPTSimpleVectorIndex.from_documents(documents)
response = index.query("kbs 연봉은 얼마?")
print(response)

In [ ]:
# 인덱싱을 파일로 저장 하고 불러옴.
from llama_index import GPTSimpleVectorIndex
index.save_to_disk('index.json')
# Load the index from your saved index.json file
index = GPTSimpleVectorIndex.load_from_disk('index.json')

In [ ]:
#---------------------------------------------------------------------------
# Q&A 예제
# 참고 : https://gpt-index.readthedocs.io/en/latest/how_to/customization/custom_prompts.html
#---------------------------------------------------------------------------
from llama_index import QuestionAnswerPrompt, GPTSimpleVectorIndex, SimpleDirectoryReader

# define custom QuestionAnswerPrompt
query_str = "가스공사가 감면해준 가스요금은?"

QA_PROMPT_TMPL = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n"
)

QA_PROMPT = QuestionAnswerPrompt(QA_PROMPT_TMPL)

# Build GPTSimpleVectorIndex
index = GPTSimpleVectorIndex.from_documents(documents)

response = index.query(query_str, text_qa_template=QA_PROMPT)
print(response)

In [ ]:
#-----------------------------------------------------------------------------
# LLM 커스터마이징 예제
#-----------------------------------------------------------------------------
from llama_index import LLMPredictor, GPTSimpleVectorIndex, PromptHelper, ServiceContext
from langchain import OpenAI

# define LLM
LLM = 'text-davinci-003'#'gpt-3.5-turbo'#'text-davinci-003'
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name=LLM))

# define prompt helper
# set maximum input size
max_input_size = 4096
# set number of output tokens
num_output = 256
# set maximum chunk overlap
max_chunk_overlap = 20
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)

index = GPTSimpleVectorIndex.from_documents(
    documents, service_context=service_context
)

response = index.query("동북아 오일허브 사업 생산유발효과는 얼마인가?")
print(response)

In [ ]:
#---------------------------------------------------------------------------
# 요약
# https://gpt-index.readthedocs.io/en/latest/use_cases/queries.html
#---------------------------------------------------------------------------
from llama_index import GPTListIndex
index = GPTListIndex.from_documents(documents)
query = '가스공사요금에 대한 내용을 요약해주세요.'
response = index.query(query, response_mode="tree_summarize")
print(response)

In [ ]:
# Flask 를 적용한 예제
# https://github.com/logan-markewich/llama_index_starter_pack/tree/main/flask_react


In [ ]:
#-------------------------------------------------------------------------------------------------------------
# 사용자 정의 LLM 모델 적용
# https://gpt-index.readthedocs.io/en/latest/how_to/customization/custom_llms.html#example-using-a-custom-llm-model
# - 에러 투성이, 속도 엄청 느림 => gpt 서버이용하는게 좋음.
#-------------------------------------------------------------------------------------------------------------

import torch
from langchain.llms.base import LLM
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex, PromptHelper
from llama_index import LLMPredictor, ServiceContext
from transformers import pipeline
from typing import Optional, List, Mapping, Any

from os import sys
sys.path.append('../BERT/')
from myutils import GPU_info
#logger = mlogging(loggername="test", logfilename="../../log/test")
device = GPU_info()
print(device)

# define prompt helper
# set maximum input size
max_input_size = 2048
# set number of output tokens
num_output = 256
# set maximum chunk overlap
max_chunk_overlap = 20
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)


class CustomLLM(LLM):
    model_name = "skt/kogpt2-base-v2"#"facebook/opt-iml-max-30b"
    pipeline = pipeline("text-generation", model=model_name, device=device, model_kwargs={"torch_dtype":torch.bfloat16})

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        prompt_length = len(prompt)
        response = self.pipeline(prompt, max_new_tokens=num_output)[0]["generated_text"]

        # only return newly generated tokens
        return response[prompt_length:]

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name_of_model": self.model_name}

    @property
    def _llm_type(self) -> str:
        return "custom"

# define our LLM

llm_predictor = LLMPredictor(llm=CustomLLM())

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)

# Load the your data
folder_path = 'data'
documents = SimpleDirectoryReader(folder_path).load_data()
index = GPTListIndex.from_documents(documents, service_context=service_context)
print(index)

In [ ]:
# Query and print response
query = '가스공사 감면 금액은?'
response = index.query(query)
print(response)